In [1]:
def makeVideo(j,initCoords,ROIsize,path,gifPath,tStart,tLength,tCadence,channels=['94','131','171','193','211','304','335']):
    plotChannel = '131'
    
    import matplotlib.colors as colors
    import astropy.units as u
    import sunpy.map
    from astropy.coordinates import SkyCoord
    import matplotlib.pyplot as plt
    from matplotlib import patches
    import time
    import numpy as np
    import PIL
    import drms
    from sunpy.util.metadata import MetaDict
    import sunpy.io.fits
    from sunpy.map import Map
    import os 
    import glob
    ### Remove old frames
    files = glob.glob(path+'*')
    for f in files:
        os.remove(f)
    
    ### Get images
    client_drms = drms.Client()
    keys_hmi,paths_hmi = client_drms.query(
        'hmi.m_45s['+tStart+'/'+tLength+'@'+tCadence+']',
        key=drms.const.all,
        seg='magnetogram'
    )

    keys_aia,paths_aia = client_drms.query(
        'aia.lev1_euv_12s['+tStart+'/'+tLength+'@'+tCadence+']['+plotChannel+']',
        key=drms.const.all,
        seg='image',
    )

    
    import astropy.time
    from astropy.visualization import ImageNormalize, SqrtStretch, time_support
    from aiapy.calibrate import correct_degradation
    from aiapy.calibrate.util import get_correction_table
    from aiapy.calibrate import register, update_pointing
    
    ### Get correction table
    import warnings
    warnings.filterwarnings('ignore')
    correction_table = get_correction_table(correction_table='/ssw/sdo/aia/response/aia_V8_20171210_050627_response_table.txt')
    import warnings
    warnings.filterwarnings('default')

    def getFrame(args):
      from sunpy.physics import solar_rotation

      i = args[0]
      paths_hmi = args[1]
      keys_hmi = args[2]
      paths_aia = args[3]
      x0 = args[4][0]
      y0 = args[4][1]
      correction_table = args[5]
      path = args[6]

      m_aia = sunpy.map.Map(paths_aia['image'][i])
      m_normalized_aia = sunpy.map.Map(
        m_aia.data/m_aia.exposure_time.to(u.s).value,
        m_aia.meta
      )
      m_updated_pointing_aia = update_pointing(m_normalized_aia)
      m_registered_aia = register(m_updated_pointing_aia)
      m_corrected_aia = correct_degradation(m_registered_aia,correction_table=correction_table)
      maps_frame_aia = m_corrected_aia
      ###
      # Adjust for shift in images and apply it to x0 and y0
      ###
      m_aia_init = sunpy.map.Map(paths_aia['image'][0])
      adjustMaps = sunpy.map.Map(m_aia_init,m_aia,sequence=True)
      adjust = sunpy.physics.solar_rotation.calculate_solar_rotate_shift(adjustMaps)
      x0n = x0 - adjust['x'][1]
      y0n = y0 - adjust['y'][1]
      ###
      bottom_left_aia = SkyCoord(x0n - ROIsize, y0n - ROIsize,
                           frame=maps_frame_aia.coordinate_frame)
      top_right_aia = SkyCoord(x0n + ROIsize, y0n + ROIsize,
                         frame=maps_frame_aia.coordinate_frame)
      submap_frame_aia = maps_frame_aia.submap(bottom_left_aia,top_right_aia)
    
      fig = plt.figure(i)
      ax = plt.subplot(projection=submap_frame_aia)
      im1 = submap_frame_aia.plot(norm=colors.Normalize(),vmin = -0.5, vmax=10)     
      ################################################################################     
      hmiPlot = 0
      try:
       dat = sunpy.io.fits.read(paths_hmi['magnetogram'][i])[1][0]
      except:
       hmiPlot = 0 
    
      if hmiPlot == 1:
          head = MetaDict({k: keys_hmi.loc[i][k] for k in keys_hmi.columns})
          m_hmi = sunpy.map.Map(dat,head)
          m_hmi = m_hmi.rotate(order=3)
          m_registered_hmi = register(m_hmi)
          maps_frame_hmi = m_registered_hmi
          bottom_left_hmi = SkyCoord(x0n - ROIsize, y0n - ROIsize,
                             frame=maps_frame_hmi.coordinate_frame)
          top_right_hmi = SkyCoord(x0n + ROIsize, y0n + ROIsize,
                             frame=maps_frame_hmi.coordinate_frame)
          submap_frame_hmi = maps_frame_hmi.submap(bottom_left_hmi,top_right_hmi)
          ### Remove noise near limb (all values off the disk)
          coord = sunpy.map.all_coordinates_from_map(submap_frame_hmi)
          r = np.sqrt(coord.Tx.value*coord.Tx.value+coord.Ty.value*coord.Ty.value)
          radsun = sunpy.map.solar_angular_radius(submap_frame_hmi.coordinate_frame).to(u.arcsec)
          mask = (r >= radsun.value).astype(np.float)
          arr = np.ma.masked_array(submap_frame_hmi.data,mask)
          ### Remove low values to reduce noise and tint
          Zm = np.ma.masked_where((arr>-20)&(arr<20),arr)
          ###
          im2 = plt.imshow(Zm,vmax=100,vmin=-100,cmap='seismic_r',alpha=.4)
            
      plt.xlabel('Helioprojective (Solar-X) [arcsec]')
      plt.ylabel('Helioprojective (Solar-Y) [arcsec]')
      if np.mean(m_aia.data)>1:
        plt.savefig(path+'frame'+str(i)+'.png',)
      plt.close(i)
        
    for i in range(len(paths_aia)):
      args = [i,paths_hmi,keys_hmi,paths_aia,initCoords,correction_table,path]
      getFrame(args)

    ### Compile gif from frames
    from PIL import Image
    from natsort import natsorted

    # filepaths
    fp_in = path+'frame*.png'
    fp_out = gifPath+'video'+str(j)+'.gif'
    img, *imgs = [Image.open(f) for f in natsorted(glob.glob(fp_in))]
    img.save(fp=fp_out, format='GIF', append_images=imgs,
    save_all=True, duration=100, loop=0)

In [2]:
###
# Intall aiapy, sunpy, graphviz, and natsort
###
import astropy.units as u
import os

#tStart = '2017-09-04T00:00:00'
tStart = '2010-09-04T00:00:00'
j = 4
initCoords = [-100* u.arcsec,-100* u.arcsec]
ROIsize = 75*u.arcsec
gifPath = '/Users/penwarden/Desktop/HEK_Videos_Paper/'
try:
    os.mkdir(gifPath)
except:
    pass
path = gifPath+'/HEK'+str(j)+'/'
try:
    os.mkdir(path)
except:
    pass
#tLength = '24h'
tLength = '5h'
tCadence = '1440s'
import warnings
warnings.filterwarnings('ignore')
makeVideo(j,initCoords,ROIsize,path,gifPath,tStart,tLength,tCadence)
import warnings
warnings.filterwarnings('default')


/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
    (-0.01124249, 7.23960506, 1.50871986e+11)>): (Tx, Ty) in arcsec
    (-25., -25.)> as keyword args. From version 2.1 passing these as positional arguments will result in an error. [sunpy.util.decorators]
    (-0.01115552, 7.24040005, 1.50875479e+11)>): (Tx, Ty) in arcsec
    (-21.0595948, -25.01314642)> as keyword args. From version 2.1 passing these as positional arguments will result in an error. [sunpy.util.decorators]
    (-0.0109465, 7.24107146, 1.50879059e+11)>): (Tx, Ty) in arcsec
    (-17.12128576, -25.02628765)> as keyword args. From version 2.1 passing these as positional arguments will result in an error. [sunpy.util.decorators]
    (-0.01061752, 7.24161373, 1.50882681e+11)>): (Tx, Ty) in arcsec
    (-13.18501762, -25.0393324)> as keyword args. From version 2.1 p